## **Final Model**
 **[Target, Result, Analysis] at the end**



In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from prettytable import PrettyTable

In [3]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       # transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                       transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values.
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])


In [4]:
#train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
#test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

train = datasets.CIFAR10('./data', train=True, download=True, transform=train_transforms)
test = datasets.CIFAR10('./data', train=False, download=True, transform=test_transforms)

100%|██████████| 170M/170M [00:45<00:00, 3.78MB/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
SEED = 1

# CUDA?
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
cuda = torch.cuda.is_available()
print("GPU Available?", device)

# For reproducibility
torch.manual_seed(SEED)

if device == "cuda":
    torch.cuda.manual_seed(SEED)
elif device == "mps":
    torch.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if device else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

# Pretty table for collecting all the accuracy and loss parameters in a table
log_table = PrettyTable()

GPU Available? mps


In [6]:
dropout_value = 0.01
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(dropout_value)
        ) # output_size = 26

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 24

         # TRANSITION BLOCK 1
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 12
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=8, kernel_size=(1, 1), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 12

        # CONVOLUTION BLOCK 2
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=12, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Dropout(dropout_value)
        ) # output_size = 10

        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=12, out_channels=12, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Dropout(dropout_value)
        ) # output_size = 8

        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=12, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 6

        # OUTPUT BLOCK
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 6

        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=6) # 7>> 9... nn.AdaptiveAvgPool((1, 1))
        ) # output_size = 1

        self.convblock8 = nn.Sequential(
            
            nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
            # nn.BatchNorm2d(10), NEVER
            # nn.ReLU() NEVER!
        ) # output_size = 1


    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)

        x = self.pool1(x)
        x = self.convblock3(x) #transition

        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        x = self.convblock7(x) #padding=1

        x = self.gap(x)
        x = self.convblock8(x) #transition
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [7]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
cuda = torch.device("cuda" if use_cuda else "cpu")
print(cuda)
model = Net().to(cuda)
summary(model, input_size=(1, 28, 28))

cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,152
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
         MaxPool2d-9           [-1, 16, 12, 12]               0
           Conv2d-10            [-1, 8, 12, 12]             128
             ReLU-11            [-1, 8, 12, 12]               0
           Conv2d-12           [-1, 12, 10, 10]             864
             ReLU-13           [-1, 12, 10, 10]               0
      BatchNorm2d-14           [-1,

In [8]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))

In [9]:
from torch.optim.lr_scheduler import StepLR

print("model running on: ", device)
log_table = PrettyTable()
log_table.field_names = ["Epoch", "Training Accuracy", "Test Accuracy", "Diff", "Training Loss", "Test Loss"]

model =  Net().to(device)
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)

EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(model, device, test_loader)
    log_table.add_row([epoch, f"{train_acc[-1]:.2f}%", f"{test_acc[-1]:.2f}%", f"{float(train_acc[-1]) - float(test_acc[-1]):.2f}" ,f"{train_losses[-1]:.4f}", f"{test_losses[-1]:.4f}"])
print(log_table)

model running on:  mps
EPOCH: 0


  0%|          | 0/391 [00:27<?, ?it/s]


RuntimeError: Given groups=1, weight of size [8, 1, 3, 3], expected input[128, 3, 32, 32] to have 1 channels, but got 3 channels instead

### TARGET
- Here we introduced StepLR learning rate scheduler. 

- Experimented with optimiser as well.

- Able to get desired result **consistently** from 7th epoch onwards, all documented as below

### RESULT

- With **StepLR and SGD**

    - Parameters            : **7864**

    - Best Train Accuracy   : **99.30%**

    - Best Test Accuracy    : **99.40%**
---
- After adding **StepLR and ADAM**

    - Parameters            : **7864**

    - Best Train Accuracy   : **99.55%**

    - Best Test Accuracy    : **99.48%**


### Analysis
- **Model Architecture**
   
    - **Channels** :  1 → 8 → 16 → MaxPool → 16 → (Transition) → 8 → 12 → 12 → 16 → 16 → GAP → 16 → Transition → 10 

    - **Receptive Field**

        | Block               | Layer      | Input Size | Kernel x Stride | Padding | Input Channel | Output Channel | Output Size | Receptive Field |
        | ------------------- | ---------- | ---------- | --------------- | ------- | ------------- | -------------- | ----------- | --------------- |
        | Input Block         | Conv1      | 28         | 3 x 1           | 0       | 1             | 8              | 26          | 3               |
        | Convolution Block 1 | Conv2      | 26         | 3 x 1           | 0       | 8             | 16             | 24          | 5               |
        | Transition Block 1  | Pool1      | 24         | 2 x 2           | 0       | 16            | 16             | 12          | 6               |
        | Transition Block 1  | Conv3      | 12         | 1 x 1           | 0       | 16            | 8              | 12          | 6               |
        | Convolution Block 2 | Conv4      | 12         | 3 x 1           | 0       | 8             | 12             | 10          | 10              |
        | Convolution Block 2 | conv5      | 10         | 3 x 1           | 0       | 12            | 12             | 8           | 14              |
        | Convolution Block 2 | Conv6      | 8          | 3 x 1           | 0       | 12            | 16             | 6           | 18              |
        | Convolution Block 2 | Conv7      | 6          | 3 x 1           | 1       | 16            | 16             | 6           | 22              |
        | Output Block        | GAP        | 6          | 3 x 1           | 0       | 16            | 16             | 1           | 28              |
        | Output Block        | Conv8      | 1          | 1 x 1           | 0       | 16            | 10             | 1           | 28              |

- **Observation**
    - After adding scheduler the model was able to achive its objective performance of 99.4% test accuracy under 15 epcoh under 800 parameters

    - Here I am giving 2 results from 2 optimisers

    ---

    - **Leraning Rate scheduler with SGD**

        
        | Epoch | Training Accuracy | Test Accuracy |  Diff | Training Loss | Test Loss |
        |-------|-------------------|---------------|-------|---------------|-----------|
        |   0   |       88.16%      |     97.17%    | -9.01 |     0.1259    |   0.1030  |
        |   1   |       97.83%      |     98.61%    | -0.78 |     0.0668    |   0.0510  |
        |   2   |       98.31%      |     98.51%    | -0.20 |     0.0186    |   0.0499  |
        |   3   |       98.45%      |     98.98%    | -0.53 |     0.0682    |   0.0353  |
        |   4   |       98.67%      |     98.97%    | -0.30 |     0.0135    |   0.0356  |
        |   5   |       98.81%      |     99.15%    | -0.34 |     0.0340    |   0.0280  |
        |   6   |       99.08%      |     99.31%    | -0.23 |     0.0057    |   0.0228  |
        |   7   |       99.17%      |     99.35%    | -0.18 |     0.0274    |   0.0219  |
        |   8   |       99.21%      |     99.29%    | -0.08 |     0.1171    |   0.0220  |
        |   9   |       99.25%      |     99.39%    | -0.14 |     0.0839    |   0.0214  |
        |   10  |       99.20%      |     99.34%    | -0.14 |     0.0466    |   0.0223  |
        |   11  |       99.23%      |     99.36%    | -0.13 |     0.0114    |   0.0212  |
        |   12  |       99.24%      |     99.37%    | -0.13 |     0.0081    |   0.0216  |
        |   13  |       99.25%      |     99.40%    | -0.15 |     0.0130    |   0.0211  |
        |   14  |       99.30%      |     99.40%    | -0.10 |     0.0250    |   0.0208  |

    ---

    - **Learning Rate scheduler with ADAM optimiser**


        | Epoch | Training Accuracy | Test Accuracy |  Diff | Training Loss | Test Loss |
        |-------|-------------------|---------------|-------|---------------|-----------|
        |   0   |       94.47%      |     97.62%    | -3.15 |     0.0283    |   0.0770  |
        |   1   |       98.11%      |     98.69%    | -0.58 |     0.0118    |   0.0396  |
        |   2   |       98.39%      |     98.63%    | -0.24 |     0.0808    |   0.0460  |
        |   3   |       98.63%      |     99.10%    | -0.47 |     0.0096    |   0.0250  |
        |   4   |       98.75%      |     98.63%    |  0.12 |     0.0096    |   0.0398  |
        |   5   |       98.84%      |     99.17%    | -0.33 |     0.0115    |   0.0267  |
        |   6   |       99.16%      |     99.41%    | -0.25 |     0.0030    |   0.0187  |
        |   7   |       99.36%      |     99.43%    | -0.07 |     0.0022    |   0.0181  |
        |   8   |       99.42%      |     99.48%    | -0.06 |     0.0168    |   0.0168  |
        |   9   |       99.44%      |     99.41%    |  0.03 |     0.0102    |   0.0187  |
        |   10  |       99.40%      |     99.46%    | -0.06 |     0.0030    |   0.0178  |
        |   11  |       99.46%      |     99.43%    |  0.03 |     0.0076    |   0.0173  |
        |   12  |       99.48%      |     99.43%    |  0.05 |     0.0025    |   0.0175  |
        |   13  |       99.51%      |     99.44%    |  0.07 |     0.0032    |   0.0172  |
        |   14  |       99.55%      |     99.45%    |  0.09 |     0.0024    |   0.0170  |

